In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import seaborn as sns


from tensorflow.keras.models import load_model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [33]:
df=pd.read_csv(r'D:\Book\bankloan.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0.0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1.0,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0.0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0.0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0.0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0.0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3.0,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1.0,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2.0,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [34]:
df=df.dropna()
df.isna().any()
df=df.drop('Loan_ID',axis=1)
df['LoanAmount']=(df['LoanAmount']*1000).astype(int)
Counter(df['Loan_Status'])


Counter({'N': 148, 'Y': 332})

In [35]:
pre_y=df['Loan_Status']
pre_X=df.drop('Loan_Status', axis=1)
dm_X=pd.get_dummies(pre_X)
dm_y=pre_y.map(dict(Y=1,N=0))
dm_X

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,1.0,4583,1508.0,128000,360.0,1.0,0,1,0,1,1,0,1,0,1,0,0
2,0.0,3000,0.0,66000,360.0,1.0,0,1,0,1,1,0,0,1,0,0,1
3,0.0,2583,2358.0,120000,360.0,1.0,0,1,0,1,0,1,1,0,0,0,1
4,0.0,6000,0.0,141000,360.0,1.0,0,1,1,0,1,0,1,0,0,0,1
5,2.0,5417,4196.0,267000,360.0,1.0,0,1,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,2900,0.0,71000,360.0,1.0,1,0,1,0,1,0,1,0,1,0,0
610,3.0,4106,0.0,40000,180.0,1.0,0,1,0,1,1,0,1,0,1,0,0
611,1.0,8072,240.0,253000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
612,2.0,7583,0.0,187000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1


In [36]:
X=dm_X
X

,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,1.0,4583,1508.0,128000,360.0,1.0,0,1,0,1,1,0,1,0,1,0,0
2,0.0,3000,0.0,66000,360.0,1.0,0,1,0,1,1,0,0,1,0,0,1
3,0.0,2583,2358.0,120000,360.0,1.0,0,1,0,1,0,1,1,0,0,0,1
4,0.0,6000,0.0,141000,360.0,1.0,0,1,1,0,1,0,1,0,0,0,1
5,2.0,5417,4196.0,267000,360.0,1.0,0,1,0,1,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,2900,0.0,71000,360.0,1.0,1,0,1,0,1,0,1,0,1,0,0
610,3.0,4106,0.0,40000,180.0,1.0,0,1,0,1,1,0,1,0,1,0,0
611,1.0,8072,240.0,253000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1
612,2.0,7583,0.0,187000,360.0,1.0,0,1,0,1,1,0,1,0,0,0,1


In [37]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 480 entries, 1 to 613
Data columns (total 17 columns):
Dependents                 480 non-null float64
ApplicantIncome            480 non-null int64
CoapplicantIncome          480 non-null float64
LoanAmount                 480 non-null int32
Loan_Amount_Term           480 non-null float64
Credit_History             480 non-null float64
Gender_Female              480 non-null uint8
Gender_Male                480 non-null uint8
Married_No                 480 non-null uint8
Married_Yes                480 non-null uint8
Education_Graduate         480 non-null uint8
Education_Not Graduate     480 non-null uint8
Self_Employed_No           480 non-null uint8
Self_Employed_Yes          480 non-null uint8
Property_Area_Rural        480 non-null uint8
Property_Area_Semiurban    480 non-null uint8
Property_Area_Urban        480 non-null uint8
dtypes: float64(4), int32(1), int64(1), uint8(11)
memory usage: 29.5 KB


In [38]:
sc = MinMaxScaler()
X=dm_X
X_test = sc.fit_transform(X)
X_test

array([[0.33333333, 0.05482993, 0.0445666 , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.03525046, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.03009276, 0.06968703, ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.33333333, 0.09798392, 0.00709283, ..., 0.        , 0.        ,
        1.        ],
       [0.66666667, 0.09193568, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.05482993, 0.        , ..., 0.        , 1.        ,
        0.        ]])

In [50]:
import pickle
from sklearn.externals import joblib
filename = 'scalers.pkl'
joblib.dump(X_test, filename)

['scalers.pkl']

In [39]:
from tensorflow.keras.models import load_model
new_model = load_model('my_model5.h5')
y_pred=new_model.predict(X_test)
y_pred=(y_pred>0.5)



In [40]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               3600      
_________________________________________________________________
dense_1 (Dense)              (None, 400)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 1604      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 85,609
Trainable params: 85,609
Non-trainable params: 0
_________________________________________________________________


In [49]:
new_model.get_weights()

[array([[ 5.85339889e-02,  5.47780097e-02, -3.54840569e-02, ...,
          2.15538967e-04,  1.80564120e-01, -1.85094118e-01],
        [-1.52641401e-01,  2.92593449e-01, -8.63487199e-02, ...,
          5.04497290e-02, -3.93526107e-01, -5.71648590e-02],
        [-2.85996459e-02,  1.16374075e-01,  3.79380584e-02, ...,
          7.52324564e-03,  3.04468334e-01, -5.70919275e-01],
        ...,
        [-2.64757514e-01,  5.00786155e-02, -7.74021223e-02, ...,
         -2.31010318e-02, -2.49715611e-01,  8.12705681e-02],
        [-2.70624757e-02, -2.96057276e-02,  3.38879637e-02, ...,
         -1.12533402e-02, -4.68631461e-02, -2.22331826e-02],
        [ 9.14768502e-02,  7.47393370e-02, -8.80353525e-02, ...,
          3.91790923e-03, -1.23547144e-01,  1.30458698e-01]], dtype=float32),
 array([-1.85934585e-02,  6.23908173e-03, -2.78032068e-02, -9.75168217e-03,
         4.74588312e-02, -9.20899026e-03, -1.59423128e-02,  0.00000000e+00,
        -9.68508795e-03, -1.58384442e-02,  2.13202015e-02, -6.

In [45]:
y_pred

array([[ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [

In [46]:
val = pd.DataFrame(y_pred, columns=['Status'])
val=val.replace({True:'Approved', False:'Rejected'})
val

,Status
0,Approved
1,Approved
2,Approved
3,Rejected
4,Approved
...,...
475,Rejected
476,Approved
477,Approved
478,Approved
